In [1]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [9]:

pip install elasticsearch 8.17.0

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement 8.17.0 (from versions: none)
ERROR: No matching distribution found for 8.17.0


In [33]:
from elasticsearch import Elasticsearch, exceptions

In [34]:
es_client = Elasticsearch('http://localhost:9200')
es_client.info()

ObjectApiResponse({'name': '4ee1bebb7ce6', 'cluster_name': 'docker-cluster', 'cluster_uuid': '6BfaagSSQumUoJRcC6nYTg', 'version': {'number': '8.17.6', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'dbcbbbd0bc4924cfeb28929dc05d82d662c527b7', 'build_date': '2025-04-30T14:07:12.231372970Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [35]:
index_settings =  {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "question": {"type": "text"}
        }
    }
}

index_name = "course-questions"

try:
    if not es_client.indices.exists(index=index_name):
        es_client.indices.create(index=index_name, body=index_settings)
        print(f"Index '{index_name}' created.")
    else:
        print(f"Index '{index_name}' already exists.")
except Exception as e:
    print(f"An Elasticsearch error occurred: {e}")

Index 'course-questions' already exists.


In [36]:
from tqdm.auto import tqdm

In [37]:
for doc in tqdm(documents): 
    es_client.index(index=index_name,document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [38]:
query = 'How do execute a command on a Kubernetes pod?'

In [ ]:
def elastic_search(query) :
    search_query = {
        "size": 4,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name,body=search_query)
    result_docs = []

    for hit in response['hits']['hits'] :
        result_docs.append(hit['_score'])
    return  result_docs
    

In [64]:
elastic_search(query)

[73.50458, 73.50458, 66.94253, 66.94253]

In [79]:
def elastic_search_filtered(query) :
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name,body=search_query)
    result_docs = []
    print(response)
    for hit in response['hits']['hits'] :
        result_docs.append(hit['_score'])
    return  result_docs
    

In [80]:
elastic_search_filtered(query = 'How do copy a file to a Docker container?')

{'took': 35, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}}


[]